In [ ]:
'''
Aim-1.using the saved model and predicting the output (i.e. segmented image)
    2.saving the segmented out to the local_path
'''

In [1]:
import cv2
import numpy as np

from matplotlib import pyplot as plt
#from patchify import patchify, unpatchify
#from PIL import Image
import segmentation_models as sm

from keras.models import load_model #loading the model 
from smooth_tiled_predictions import predict_img_with_smooth_windowing #this is use to smothing and blending of segmented images
from sklearn.preprocessing import MinMaxScaler #scaliing and preprocessing of input image

Segmentation Models: using `keras` framework.


In [2]:
scaler = MinMaxScaler() #declaring the object for scaling purpose

BACKBONE = 'resnet34' #declaration of base network of resnet34
preprocess_input = sm.get_preprocessing(BACKBONE)

In [3]:
#Reading the image
img = cv2.imread(r"C:\Python310\tf\Scripts\Project_main\landcover.ai.v1\Result\image_s1.tif")  
#scaling the input image and reshapeing it(resizing it)
input_img = scaler.fit_transform(img.reshape(-1, img.shape[-1])).reshape(img.shape)
input_img = preprocess_input(input_img)

original_mask = cv2.imread(r"C:\Python310\tf\Scripts\Project_main\landcover.ai.v1\masks\M-34-6-A-d-2-2.tif")
original_mask = original_mask[:,:,0]  #Use only single channel...
#original_mask = to_categorical(original_mask, num_classes=n_classes)

In [4]:
model = load_model("landcover_2_epochs_RESNET_backbone_batch16.hdf5", compile=False) #loading the model
                  
patch_size =256 # size of patches
n_classes = 5 # Number of classes 

In [5]:
predictions_smooth = predict_img_with_smooth_windowing( 
    input_img,
    window_size=patch_size,
    subdivisions=2,  # Minimal amount of overlap for windowing. Must be an even number.
    nb_classes=n_classes,
    pred_func=(lambda img_batch_subdiv: model.predict((img_batch_subdiv))
    )
)

final_prediction = np.argmax(predictions_smooth, axis=2)
#MemoryError: Unable to allocate 7.48 GiB for an array with shape (74, 69, 256, 256, 3) and data type float64

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

20/20 [==============================] - 57s 3s/step


 12%|██████████▌                                                                         | 1/8 [01:02<07:19, 62.83s/it]

20/20 [==============================] - 61s 3s/step


 25%|█████████████████████                                                               | 2/8 [02:10<06:35, 65.93s/it]

...

 75%|███████████████████████████████████████████████████████████████                     | 6/8 [06:34<02:11, 65.59s/it]

20/20 [==============================] - 58s 3s/step


 88%|█████████████████████████████████████████████████████████████████████████▌          | 7/8 [07:38<01:05, 65.14s/it]

20/20 [==============================] - 57s 3s/step


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [08:42<00:00, 65.34s/it]


In [14]:
#Save prediction and original mask for comparison
#"C:\Python310\tf\Scripts\Project_main\landcover.ai.v1\Result"
plt.imsave(r"C:\Python310\tf\Scripts\Project_main\landcover.ai.v1\Result\R2.tiff",final_prediction)         
#plt.imsave('data/test_images/N-34-66-C-c-4-3.tif_mask.jpg', original_mask)

In [ ]:
#This block of code is to represent the originalimages, original mask,and predicted segmented images side by side to compare it
plt.figure(figsize=(12, 12))
plt.subplot(221)
plt.title('Testing Image')
plt.imshow(img)
plt.subplot(222)
plt.title('Testing Label')
plt.imshow(original_mask)
plt.subplot(223)
plt.title('Prediction with smooth blending')
plt.imshow(final_prediction)
plt.show()